In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
BIT_CNT_EXT = 14

In [3]:
df = pd.read_csv("out.csv")
df.head()

,angle,sin,cos
0,0,0,4000
1,1,1,3fff
2,2,2,3fff
3,3,3,3fff
4,4,3,3fff


In [4]:
def ToDouble(x):
    result = 0.0
    x = bin(int(x, 16))[2:].zfill(BIT_CNT_EXT+2)
    temp = 1.0
    for bit in x[1:]:
        if bit == '1':
            result += temp
        temp = temp/2
    if x[0] == '1':
        result -= 2
    return result

for col in df:
    df["raw_" + str(col)] = df[col]
    df[col] = df[col].apply(ToDouble)
df.head()

,angle,sin,cos,raw_angle,raw_sin,raw_cos
0,0.000000,0.000000,1.000000,0,0,4000
1,0.000061,0.000061,0.999939,1,1,3fff
2,0.000122,0.000122,0.999939,2,2,3fff
3,0.000183,0.000183,0.999939,3,3,3fff
4,0.000244,0.000183,0.999939,4,3,3fff


In [5]:
def errF(angle, val, fun):
    ULP = 2**-BIT_CNT_EXT
    temp = fun(angle)
    err = np.abs(val - temp)
    ulpErr = err/ULP
    return ulpErr

df['ErrSin[ULP]'] = df.apply(lambda x : errF(x.angle, x.sin, np.sin), axis=1)
df['ErrCos[ULP]'] = df.apply(lambda x : errF(x.angle, x.cos, np.cos), axis=1)
df.head()

,angle,sin,cos,raw_angle,raw_sin,raw_cos,ErrSin[ULP],ErrCos[ULP]
0,0.000000,0.000000,1.000000,0,0,4000,0.000000e+00,0.000000
1,0.000061,0.000061,0.999939,1,1,3fff,6.208817e-10,0.999969
2,0.000122,0.000122,0.999939,2,2,3fff,4.967054e-09,0.999878
3,0.000183,0.000183,0.999939,3,3,3fff,1.676381e-08,0.999725
4,0.000244,0.000183,0.999939,4,3,3fff,1.000000e+00,0.999512


In [6]:
df[['ErrSin[ULP]', 'ErrCos[ULP]']].describe(include='all')

,ErrSin[ULP],ErrCos[ULP]
count,25735.000000,25735.000000
mean,0.513800,0.511172
std,0.293323,0.290555
min,0.000000,0.000000
25%,0.259658,0.258500
50%,0.517638,0.515402
75%,0.770506,0.768069
max,1.000000,0.999972
